In [1]:
!pip install -U sentence-transformers 

from sentence_transformers import SentenceTransformer
import pandas as pd
from time import time

decoded_df2=pd.read_csv('decoded_df2__l1.csv')

cpcs_nums = decoded_df2.cpsc_case_number

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=ad5a082f204a4d4c4c7080355eb97b91779ead3505be8b60f3201a8fa691831e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


/tmp/ipykernel_20/3672798480.py:7: DtypeWarning: Columns (9,12,13,15,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  decoded_df2=pd.read_csv('/kaggle/input/neiss-sentence-transform-embeddings/decoded_df2__l1.csv')


In [2]:
decoded_df2['narrative'] 

0         94YOM FELL TO THE FLOOR AT THE NURSING HOME ON...
1         86YOM FELL IN THE SHOWER AT HOME AND SUSTAINED...
2         87YOF WAS GETTING UP FROM THE COUCH AND FELL T...
3         67YOF WAS AT A FRIENDS HOUSE AND SLIPPED ON WA...
4         70YOF WAS STANDING ON A STEP STOOL AND FELL OF...
                                ...                        
533513    77YOF TRIPPED OVER CARPET AND FELL. DX: CLOSED...
533514    82YOM WAS BACKING UP AND TRIPPED OVER A CORD A...
533515    87YOM WAS GOING UP A FLIGHT OF STAIRS AND LOST...
533516    72 YOM PRESENTS FROM HOME  WITH COMPLAINT OF H...
533517    81 YOF PRESENTS FROM HOME COMPLAINING OF 2 FAL...
Name: narrative, Length: 533518, dtype: object

In [3]:
# define the document embedding models to use for comparison
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

import tensorflow_hub as hub
model_use = hub.load(module_url)
model_st1 = SentenceTransformer('all-mpnet-base-v2')
model_st2 = SentenceTransformer('all-MiniLM-L6-v2')
model_st3 = SentenceTransformer('paraphrase-mpnet-base-v2')  # already explored
# paraphrase-multilingual-mpnet-base-v2")

def embed(model, model_type, sentences):
    """
    wrapper function for generating message embeddings
    """ 
    if model_type == 'use':
        embeddings = model(sentences)
    elif model_type == 'sentence transformer':
        embeddings = model.encode(sentences)

    return embeddings

import pickle

Embeddings={}
narratives = list( decoded_df2['narrative_cleaned'] )
with open( f"all_cpsc.pkl", 'wb') as handle:
    pickle.dump( {'cpsc': cpcs_nums } , handle)

    
for emb in [3,4,]:
    starttime=time()

    if emb==1: # 768
        model_st1 = SentenceTransformer('all-mpnet-base-v2')
        Embeddings[emb] = embed(model_st1, 'sentence transformer', narratives )
    elif emb==2: # 384
        model_st2 = SentenceTransformer('all-MiniLM-L6-v2')
        Embeddings[emb] = embed(model_st2, 'sentence transformer', narratives )
    elif emb==3: #768
        model_st3 = SentenceTransformer('paraphrase-mpnet-base-v2')  # already explored paraphrase-multilingual-mpnet-base-v2")
        Embeddings[emb] = embed(model_st3, 'sentence transformer', narratives )
    elif emb==4: # 512
        model_use = hub.load(module_url)
        Embeddings[emb] = embed(model_use, 'use', narratives )

    print(Embeddings[emb].shape[1],'dimensions')
    with open( f"all_embeddings_{emb}.pkl", 'wb') as handle:
        pickle.dump( {"embeddings": Embeddings[emb] } , handle)

    print( emb, (starttime - time())/60 )

Batches:   0%|          | 0/16673 [00:00<?, ?it/s]

768 dimensions
3 -261.5387110829353
512 dimensions
4 -1.6876668810844422
